In [ ]:
import pandas as pd
import numpy as np
import plotly_express as px
import seaborn as sns 
import matplotlib.pyplot as plt  
import warnings 
from matplotlib.pylab import rcParams

rcParams['figure.figsize'] = [10,5]

suicides = pd.read_csv("../input/suicide-rates-overview-1985-to-2016/master.csv")
del suicides['HDI for year']
del suicides['country-year']


iso_country_map = pd.read_csv("../input/countries-iso-codes/wikipedia-iso-country-codes.csv")
iso_country_map = iso_country_map.rename(columns = {'English short name lower case':"country"})


concap =pd.read_csv("../input/country-to-continent/countryContinent.csv", encoding='iso-8859-1')[['code_3', 'continent', 'sub_region']]
concap = concap.rename(columns = {'code_3':"Alpha-3 code"})

correct_names = {'Cabo Verde': 'Cape Verde', 'Macau': 'Macao', 'Republic of Korea': "Korea, Democratic People's Republic of" , 
 'Russian Federation': 'Russia',
 'Saint Vincent and Grenadines':'Saint Vincent and the Grenadines' 
 , 'United States': 'United States Of America'}

def correct_country(x):
    if x in correct_names:
        return correct_names[x]
    else:
        return x

suicides['country'] = suicides['country'].apply(lambda x : correct_country(x))

suicides = pd.merge(suicides,iso_country_map,on='country',how='left')
suicides = pd.merge(suicides,concap,on='Alpha-3 code',how='left')
suicides['gdp'] = suicides['gdp_per_capita ($)']*suicides['population']

In [ ]:
suicides

In [ ]:
suicides.head()

In [ ]:
df1= suicides.copy()
df1.info()


# 1- Global Suicides(per 100K)-trend over time 1985-2016

In [ ]:
plt.figure(figsize=(10,7))
sns.stripplot(x="year",y='suicides/100k pop',data=df1)
plt.xticks(rotation=45)
plt.show()

# 2- Global Suicides(per 100K) by Continent

In [ ]:
plt.figure(figsize=(10,7))
sns.stripplot(x="continent",y='suicides/100k pop',data=df1)
plt.xticks(rotation=45)
plt.show()

# 3-Global Suicides(per 100k) by Gender and trend over time 1985-2016

In [ ]:
plt.figure(figsize=(10,7))
sns.stripplot(x="sex",y='suicides/100k pop',data=df1)
plt.xticks(rotation=45)
plt.show()

# 4-Population-gdp_per_capita Plot

In [ ]:
px.scatter(df1,x = 'gdp_per_capita ($)', y = 'population')


# 5- Correlation between GDP(per Capita) and suicides per 100k

In [ ]:
sns.FacetGrid(df1,hue='year',size=15).map(plt.scatter,'gdp_per_capita ($)','suicides/100k pop').add_legend()
plt.show()

In [ ]:
rcParams['figure.figsize'] = [10,5]
sns.heatmap(df1.corr(), annot=True, linewidths=0.2, cmap='coolwarm' )
plt.title('Correlation between GDP(per Capita) and suicides per 100k', size=15, fontweight='bold') ;
plt.xticks(rotation=45)

# 6- Generation hue Gender Counter

In [ ]:
f,ax=plt.subplots(1,2,figsize=(18,8))
df1['generation'].value_counts().plot.pie(explode=[0.1,0.1,0.1,0.1,0.1,0.1],autopct='%1.1f%%',ax=ax[0],shadow=True)
ax[0].set_title('Generations Count')
ax[0].set_ylabel('count')
sns.countplot('generation',data=df1,ax=ax[1])
ax[1].set_title('Generations Count')
plt.show()

# 7- Which age of people suicide a most

In [ ]:
index_population=[]
for age in df1['age'].unique():
    index_population.append(sum(df1[df1['age']==age].population)/len(df1[df1['age']==age].population))
    
plt.bar(['15-24 years','35-54 years','75+ years','25-34 years','55-74 years','5-14 years'],index_population,align='center',alpha=0.5)
plt.xticks(rotation=90)
plt.show()

# 8-Which generation of people suicide a most

In [ ]:
px.scatter(df1,x = 'generation', y = 'population')